# Load data
_Note_: tempo = bpm

Load the V, A, D dimension values from the original MuSe dataset

In [ ]:
import pandas as pd

orig_dataset = pd.read_csv("datasets/muse_v3.csv", nrows=5)
orig_dataset.columns

In [ ]:
import pandas as pd

KEEP_COLS_MASTER = ["name", "album", "artist", "release_date", "length",
             "tempo", "key", "time_signature",
             "mood"]

KEEP_COLS_MODEL = ["name", "release_date", "length", "tempo",
                   "key", "time_signature", "mood"]

# songs_df = pd.read_csv("data_moods.csv", usecols=KEEP_COLS_MASTER)
# make names the indices
songs_df = pd.read_csv("data_moods.csv", usecols=KEEP_COLS_MODEL, parse_dates=True, infer_datetime_format=True)
songs_df.set_index("name", inplace=True)
songs_df

Convert dates to numbers, specfically seconds.

_Note_: This goddamn dataset only gives the year for some songs.

In [ ]:
print(songs_df[songs_df["release_date"]=="1993"])
# credit https://stackoverflow.com/a/54312941
songs_df["release_date"] = pd.to_datetime(songs_df["release_date"], format="mixed").astype(int)/ 10**9
songs_df["release_date"]

Model will take data as (`X_train`,`y_train`)
- `X_train` is a 2D NumPy array of shape `n_samples` x `n_features`, i.e. one song per row, one metadata feature per column
- `y_train` is just convert the "mood" column into a 1D NumPy array

In [36]:
# How much should we take for training, testing?
songs_df.index.size

0.7 * songs_df.index.size

480.2

In [37]:
import numpy as np
from sklearn.model_selection import train_test_split

# use a constant random state for reproducibility
X_train, X_test, y_train, y_test = train_test_split(songs_df.drop(columns=["mood"]), songs_df[["mood"]],
                                                    test_size=0.33, random_state=42)

Make the model
Use [a i[stogram-Based Gradient Boosted Tree Classifier.
](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.HistGradientBoostingClassifier.html#sklearn.ensemble.HistGradientBoostingClassifier.fit)

In [61]:
from sklearn.ensemble import HistGradientBoostingClassifier

hgbt = HistGradientBoostingClassifier(max_iter=90, max_bins=30)
classification = hgbt.fit(X_train, y_train)

/home/tom/miniconda3/envs/cmpt353/lib/python3.10/site-packages/sklearn/utils/validation.py:1229: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [62]:
classification.score(X_test, y_test) 

0.6519823788546255